In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

In [3]:
data = pd.read_csv('cleaned_data1.csv')
print(data.head())
print(data.info())

   weight  aki_stage  delay_rrt  gender  admission_age  ethnicity  \
0  104.50          1          1       1      66.262081          0   
1  100.00          1          1       1      70.489938          0   
2   98.95          3          0       1      37.700917          0   
3  105.70          3          1       0      59.967156          1   
4   69.00          1          0       1      79.060470          1   

   hematocrit_min  hematocrit_max  hemoglobin_min  hemoglobin_max  ...  \
0            31.4            31.9            11.3            11.3  ...   
1            34.0            34.2            11.3            11.4  ...   
2            27.1            29.4             9.9            10.4  ...   
3            25.0            30.9             8.1            10.1  ...   
4            38.0            38.0            13.2            13.2  ...   

   ptt_max  heart_rate_mean    sbp_mean  dbp_mean   mbp_mean  resp_rate_mean  \
0     47.5        89.625000  132.500000   68.5000  83.090909

In [4]:
# Data preprocessing
X = data.drop('aki_stage', axis=1).values
y = data['aki_stage'].values

scaler = StandardScaler()
X = scaler.fit_transform(X)
y = data['aki_stage'].values - 1

In [5]:
# Separate training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

In [61]:
class DNN(nn.Module):
    def __init__(self, input_dim):
        super(DNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.fc2 = nn.Linear(128, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, 3)
        self.dropout = nn.Dropout(0.5)



    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = self.fc3(x)
        return x



In [62]:
model = DNN(input_dim=X_train.shape[1])
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

In [63]:
# Training model
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    scheduler.step()
    
    model.eval()
    with torch.no_grad():
        outputs = model(X_test)
        _, predicted = torch.max(outputs.data, 1)
        accuracy = accuracy_score(y_test, predicted)
        f1 = f1_score(y_test, predicted, average='weighted')
        
        # 计算AUC
        probabilities = torch.softmax(outputs, dim=1).cpu().numpy()
        auc = roc_auc_score(y_test.cpu().numpy(), probabilities, multi_class='ovr')
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}, AUC: {auc:.4f}')

Epoch [1/100], Accuracy: 0.1955, F1 Score: 0.0733, AUC: 0.5425
Epoch [2/100], Accuracy: 0.2235, F1 Score: 0.1434, AUC: 0.5480
Epoch [3/100], Accuracy: 0.3352, F1 Score: 0.3290, AUC: 0.5510
Epoch [4/100], Accuracy: 0.4358, F1 Score: 0.4525, AUC: 0.5551
Epoch [5/100], Accuracy: 0.5196, F1 Score: 0.5231, AUC: 0.5552
Epoch [6/100], Accuracy: 0.5782, F1 Score: 0.5624, AUC: 0.5521
Epoch [7/100], Accuracy: 0.6257, F1 Score: 0.5878, AUC: 0.5499
Epoch [8/100], Accuracy: 0.6425, F1 Score: 0.5956, AUC: 0.5476
Epoch [9/100], Accuracy: 0.6648, F1 Score: 0.6071, AUC: 0.5463
Epoch [10/100], Accuracy: 0.6676, F1 Score: 0.6033, AUC: 0.5445
Epoch [11/100], Accuracy: 0.6704, F1 Score: 0.6001, AUC: 0.5431
Epoch [12/100], Accuracy: 0.6788, F1 Score: 0.5992, AUC: 0.5427
Epoch [13/100], Accuracy: 0.6872, F1 Score: 0.6013, AUC: 0.5426
Epoch [14/100], Accuracy: 0.6872, F1 Score: 0.6013, AUC: 0.5427
Epoch [15/100], Accuracy: 0.6872, F1 Score: 0.5979, AUC: 0.5431
Epoch [16/100], Accuracy: 0.6872, F1 Score: 0.597

In [64]:
# evaluation model
model.eval()
with torch.no_grad():
    outputs = model(X_test)
    _, predicted = torch.max(outputs.data, 1)
    accuracy = accuracy_score(y_test, predicted)
    f1 = f1_score(y_test, predicted, average='weighted')
    
    # 计算最终的AUC
    probabilities = torch.softmax(outputs, dim=1).cpu().numpy()
    auc = roc_auc_score(y_test.cpu().numpy(), probabilities, multi_class='ovr')

    print(f'Final Accuracy: {accuracy:.4f}')
    print(f'Final F1 Score: {f1:.4f}')
    print(f'Final AUC: {auc:.4f}')

Final Accuracy: 0.7095
Final F1 Score: 0.5987
Final AUC: 0.5430
